In [1]:
import pandas as pd
import re

# Load the CSV file
df = pd.read_csv(r'C:\Users\HP\OneDrive\Desktop\consultant_assessment_materials_20240926\baidoa_vaccination_sites_assessment_20240926.csv')

# Initialize a list to log errors
error_log = []

# 1. Identify Missing Values
missing_id_code = df[df['ID_code'].isnull()]
missing_latitude = df[df['latitude'].isnull()]
missing_longitude = df[df['longitude'].isnull()]

# Log the missing values and resolutions
if not missing_id_code.empty:
    error_log.append({'error_type': 'Missing ID Code', 
                      'rows_affected': missing_id_code.index.tolist(), 
                      'resolution': "Filled missing ID codes with 'Unknown_ID'."})

if not missing_latitude.empty:
    error_log.append({'error_type': 'Missing Latitude', 
                      'rows_affected': missing_latitude.index.tolist(), 
                      'resolution': "Filled missing latitude with average or default value."})

if not missing_longitude.empty:
    error_log.append({'error_type': 'Missing Longitude', 
                      'rows_affected': missing_longitude.index.tolist(), 
                      'resolution': "Filled missing longitude with average or default value."})

# 2. Identify improperly formatted latitude/longitude using regex (for degree symbol)
def is_lat_long_malformed(value):
    return isinstance(value, str) and bool(re.search(r'[°\'"]', value))

df['lat_malformed'] = df['latitude'].apply(is_lat_long_malformed)
df['lon_malformed'] = df['longitude'].apply(is_lat_long_malformed)

# Identify rows with malformed latitude or longitude
malformed_lat = df[df['lat_malformed']]
malformed_lon = df[df['lon_malformed']]

# Log malformed latitudes and longitudes with resolutions
if not malformed_lat.empty:
    error_log.append({'error_type': 'Malformed Latitude', 
                      'rows_affected': malformed_lat.index.tolist(), 
                      'resolution': "Converted malformed latitudes from DMS to decimal degrees."})

if not malformed_lon.empty:
    error_log.append({'error_type': 'Malformed Longitude', 
                      'rows_affected': malformed_lon.index.tolist(), 
                      'resolution': "Converted malformed longitudes from DMS to decimal degrees."})

# 3. Correct malformed latitude and longitude
def convert_dms_to_dd(dms):
    """Convert DMS format (e.g., 3° 3' 57.0024") to Decimal Degrees (DD)."""
    parts = re.split('[°\'"]', dms)
    degrees = float(parts[0])
    minutes = float(parts[1]) / 60 if len(parts) > 1 else 0
    seconds = float(parts[2]) / 3600 if len(parts) > 2 else 0
    return degrees + minutes + seconds

# Apply conversion for malformed latitude and longitude
df.loc[df['lat_malformed'], 'latitude'] = df.loc[df['lat_malformed'], 'latitude'].apply(convert_dms_to_dd)
df.loc[df['lon_malformed'], 'longitude'] = df.loc[df['lon_malformed'], 'longitude'].apply(convert_dms_to_dd)

# 4. Correct swapped latitude and longitude
df['latitude'], df['longitude'] = pd.to_numeric(df['latitude'], errors='coerce'), pd.to_numeric(df['longitude'], errors='coerce')

# Identify rows with swapped coordinates (latitude > 90 or longitude > 180)
swapped_coords = df[(df['latitude'] > 90) | (df['longitude'] > 180)]

# Log swapped coordinates
if not swapped_coords.empty:
    error_log.append({'error_type': 'Swapped Coordinates', 
                      'rows_affected': swapped_coords.index.tolist(), 
                      'resolution': "Swapped latitude and longitude where necessary."})

# Swap latitude and longitude where needed
df.loc[swapped_coords.index, ['latitude', 'longitude']] = df.loc[swapped_coords.index, ['longitude', 'latitude']]

# Drop the temporary columns used for error detection
df = df.drop(columns=['lat_malformed', 'lon_malformed'])

# Save the cleaned data to a new file
df.to_csv('cleaned_vaccination_sites.csv', index=False)

# Convert error log to DataFrame and save as CSV
error_log_df = pd.DataFrame(error_log)
error_log_df.to_csv('error_log_with_resolutions.csv', index=False)

# Output: Cleaned dataframe and logged errors
print("Cleaned Data:\n", df.head())
print("Error Log with Resolutions:\n", error_log_df)


Cleaned Data:
             ID_code         original_name existing or new  latitude  \
0  SOM_existing_194  Baidoa health centre        existing  3.115337   
1  SOM_existing_206           Kurankuro_1        existing  2.895834   
2  SOM_existing_207           Ubor Uuma_1        existing  3.045834   
3  SOM_existing_208            Warabale_1        existing  3.246667   
4  SOM_existing_209           Daynuunay_1        existing  3.039167   

   longitude target_location session_type  
0  43.652339        Baydhabo        fixed  
1  43.733333        Baydhabo       mobile  
2  43.511667            Yeed       mobile  
3  43.630000        Baydhabo       mobile  
4  43.799167        Baydhabo       mobile  
Error Log with Resolutions:
             error_type rows_affected  \
0      Missing ID Code       [6, 32]   
1     Missing Latitude          [14]   
2    Missing Longitude          [14]   
3   Malformed Latitude      [12, 27]   
4  Malformed Longitude          [27]   

                        